In [1]:
!pip install pytelegrambotapi
!ls -la
import telebot
from telebot import types
import logging
import gc

     |████████████████████████████████| 102kB 8.9MB/s 
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-3.8.1-cp37-none-any.whl size=75902 sha256=15bbcd395f45617eab5d9c11e31cf9718a198ebc3740fd44febf2d73f5720638
  Stored in directory: /root/.cache/pip/wheels/df/97/1c/6bf3934203f0673ac6c3e6aecb0bead54f95df437f492f5d4e
Successfully built pytelegrambotapi
total 16
drwxr-xr-x 1 root root 4096 Jul  1 13:42 .
drwxr-xr-x 1 root root 4096 Jul 13 19:23 ..
drwxr-xr-x 4 root root 4096 Jul  1 13:41 .config
drwxr-xr-x 1 root root 4096 Jul  1 13:42 sample_data


In [2]:
import random

def format_point(x):
  if x == 1337:
    return " "
  if x == 1:
    return "X"
  return "0"

class Crosses_zeros():
  def __init__(self):
    self.table = [1337 for i in range(9)]
    self.player_now = 1

  def winner_positions(self):
    return [[0, 3, 6], [1, 4, 7], [2, 5, 8], [0, 1, 2], [3, 4, 5], [6, 7, 8], [0, 4, 8], [2, 4, 6]]
  
  def do_action(self, action):
    self.table = action
    self.player_now = 1 - self.player_now

  def action_list(self):
    actions = []
    for i in range(9):
      if (self.table[i] == 1337):
        actions.append(self.table[:i] + [self.player_now] + self.table[i + 1:])
    return actions
  
  def playable(self):
    return self.check_winner() == "c"

  def check_winner(self):
    sum_now = 0
    for combination in self.winner_positions():
      for pos in combination:
        sum_now += self.table[pos]
      if sum_now == 0:
        return "0"
      if sum_now == 3:
        return "X"
      sum_now = 0
    if any(self.action_list()):
      return "c"
    return "-"


  def print_table(self):
    for i in range(9):
      print(format_point(self.table[i]), end=("\n" * 0**(2 - i % 3) + " " * (1 - 0**(2 - i % 3))))
    print()

  def table_to_string(self):
    a = "<code>"
    for i in range(5):
      for j in range(5):
        if (i % 2 == 1) and (j % 2 == 1):
          a += '+'
        elif (i % 2 == 1):
          a += '-'
        elif (j % 2 == 1):
          a += "|"
        else:
          a += format_point(self.table[(i // 2) * 3 + j // 2])
      a += '\n'
    a += "</code>"
    return a
   


In [3]:
class Agent007():
  def __init__(self, game, gamma=0.5, rand=0.1, player=1):
    self.game = game
    self.gamma = gamma
    self.rand = rand
    self.player = player
    self.V_P = dict()
  
  def value(self, state):
    return self.V_P.get(tuple(state), 0.0)

  def best_action_now(self, game, func):
    good_actions = []
    tmp = self.value(game.action_list()[0])
    for action in game.action_list():
      tmp = func(tmp, self.value(action))
    for action in game.action_list():
      if self.value(action) == tmp:
        good_actions.append(action)
    return good_actions
  

  def learn(self, count_of_games):
    for i in range(count_of_games):
      now_game = self.game()
      tmp, action = self.best_actions(now_game)
      while now_game.playable():
        action = self.action_with_learn(now_game, action)
    
  def select_action(self, game):
    if self.player == game.player_now:
      return random.choice(self.best_action_now(game, max))
    return random.choice(self.best_action_now(game, min))

  def best_actions(self, game):
    best_action = self.select_action(game)
    selected_action = best_action
    if random.random() < self.rand:
      selected_action = random.choice(game.action_list())
    return best_action, selected_action


  def reward(self, game):
    if game.check_winner() == format_point(self.player): #если выиграли
      return 1
    elif game.check_winner() == '-' or game.check_winner() == "c": #если ничья
      return 0  
    else: #если проиграли
      return -1 


  def action_with_learn(self, game, action):
    game.do_action(action)
    r = self.reward(game) #получили какой-то реворд 
    random_next_action = 0 #вознагрождение от след состояния пока 0
    next_state = 0 #след действие еще не выбрали
    best_next_action = 0
    if game.playable():
      best_next_action, random_next_action = self.best_actions(game) #дописать функцию, где будет еще рандом(иначе зациклимся)
      next_state = self.value(best_next_action) #берем САМОЕ лучшее состояние
    self.V_P[tuple(action)] = self.value(action) + self.gamma * (r + self.gamma * next_state) 
    #V(s) = Е(R_{t} + gamma*G_{t})  /// где G_{t} = reward + gamma*G_{t+1}
    #(V(s) = pi(a|s) + p(r, s'|s, a)[r + gamma*v(s')])
    return random_next_action #возвращаем рандомное действие


  def game_now(self, to_print=False):
    now_game = self.game()
    while now_game.playable():
      now_game.do_action(self.select_action(now_game))
      if to_print:
        now_game.print_table()
    winner = now_game.check_winner()
    if to_print:
      if winner == "0":
        print("Выиграли нолики")
      elif winner == "X":
        print("Выиграли крестики")
      else:
        print("Ничья! Давайте жить дружно!!!")
    return now_game.check_winner()

  def select_action_for_interactive_game(self, game, player):
      if player == 1:
        return random.choice(self.best_action_now(game, max))
      return random.choice(self.best_action_now(game, min))

In [9]:
def game_st(agent):
  results = [agent.game_now() for i in range(10000)]
  game_stats = {k: results.count(k)/100 for k in ["X", "0", '-']}
  print("Статистика обучения: {}".format(game_stats))

def to_numb(x):
  x = x.lower()
  if len(x) != 2 or x[1].isalpha() or int(x[1]) > 3 :
    return 10
  if x[0] == "a":
    return int(x[1]) - 1
  if x[0] == "b":
    return 2 + int(x[1])
  if x[0] == "c":
    return 5 + int(x[1])
  return 10

In [5]:
agent = Agent007(Crosses_zeros)
print('Before:')
game_st(agent)

agent.learn(50000)
print('After:')
game_st(agent)

Before:
Статистика обучения: {'X': 57.86, '0': 29.15, '-': 12.99}
After:
Статистика обучения: {'X': 0.0, '0': 0.0, '-': 100.0}


In [10]:
player = dict()
game = dict()


bot = telebot.TeleBot('1818851136:AAHzS5ZKjx_GNfe5V_LhpdnB95lWlXW9E08')
@bot.message_handler(content_types=['text'])
def get_text_messages(message):
  if message.text.lower() == "привет":
    bot.send_message(message.from_user.id, "Привет, хочешь поиграть?")
    bot.register_next_step_handler(message, continue_bot);
  elif message.text == "/help":
    bot.send_message(message.from_user.id, "Напиши привет")
  else:
    bot.send_message(message.from_user.id, "Я тебя не понимаю. Напиши /help.")

def continue_bot(message):
  if (message.text).lower() == 'да':
    figure(message)
  else:
    bot.send_message(message.from_user.id, "Тогда пока!")
      
def figure(message):
      keyboard = types.InlineKeyboardMarkup(); #наша клавиатура
      key_yes = types.InlineKeyboardButton(text='X', callback_data="1"); #кнопка «Да»
      keyboard.add(key_yes); #добавляем кнопку в клавиатуру
      key_yes2 = types.InlineKeyboardButton(text='0', callback_data="0");
      keyboard.add(key_yes2);
      question = 'Что ты выберешь?';
      bot.send_message(message.from_user.id, text=question, reply_markup=keyboard)

@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    bot.send_message(call.message.chat.id, 'Хороший выбор! Если хочешь, чтобы я тебя понял, свой ход делай с помощью команд: a1, a2, a3, b1, b2, b3, c1, c2, c3, где a, b, c - строки, а 1, 2, 3 - столбцы ')
    global player, game
    game[call.message.chat.id] = Crosses_zeros()
    player[call.message.chat.id] = int(call.data)
    if  player[call.message.chat.id] == 0:
        game[call.message.chat.id].do_action(agent.select_action_for_interactive_game(game[call.message.chat.id], 1))
        bot.send_message(call.message.chat.id, game[call.message.chat.id].table_to_string(), parse_mode="HTML")
    bot.register_next_step_handler(call.message, agent_play);

def agent_play(message):
    global player, game
    now = to_numb(message.text)
    table = game[message.chat.id].table
    if now < 0 or now > 8 or table[now] != 1337:
      bot.reply_to(message, "Некорректный ход")
    else:
      table[now] = player[message.chat.id]
      game[message.chat.id].do_action(table)
      if game[message.chat.id].playable():
        game[message.chat.id].do_action(agent.select_action_for_interactive_game(game[message.chat.id], 1 - player[message.chat.id]))
      bot.send_message(message.chat.id, game[message.chat.id].table_to_string(), parse_mode="HTML")
    if game[message.chat.id].playable():
      bot.register_next_step_handler(message, agent_play)
    else:
      if game[message.chat.id].check_winner() == 'X':
        bot.send_message(message.chat.id, 'Выиграл X!!!', parse_mode="HTML")
      elif game[message.chat.id].check_winner() == '0':
        bot.send_message(message.chat.id, 'Выиграл 0!!!', parse_mode="HTML")
      else:  
        bot.send_message(message.chat.id, 'Ничья!!!', parse_mode="HTML")
    

bot.polling(none_stop=True, interval=0)  
